In [ ]:
import cv2
import glob
import imutils
import numpy as np
import os
import pickle
import picklefrom imutils import paths
from imutils import paths
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.models import Sequential
from keras.src.callbacks import History
from PIL import Image
from PIL import ImageFilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# TRATAR IMAGEM

def prepare_image(img):
    img = img.filter(ImageFilter.SMOOTH_MORE)
    img = img.filter(ImageFilter.SMOOTH_MORE)
    if 'L' != img.mode:
        img.convert("L")
    return img

def remove_noise(img, pass_factor):

    for column in range(img.size[0]):
        for line in range(img.size[1]):
            value = remove_noise_by_pixel(img, column, line, pass_factor)
            img.putpixel((column, line), value)

    return img

def remove_noise_by_pixel(img, column, line, pass_factor):

    if img.getpixel((column, line)) < pass_factor:
        return (0)
    return (255)

def tratar_imagem(noise, arquivo_origem, arquivo_destino="tratado"):

    arquivos = glob.glob(f'{arquivo_origem}/*')

    for arquivo in arquivos:
        imagem = cv2.imread(arquivo)
        imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
        _, imagem_tratada=  cv2.threshold(imagem_cinza,127,255, cv2.THRESH_BINARY or cv2.THRESH_OTSU)
        nome_arquivo = os.path.basename(arquivo)
        cv2.imwrite(f"{arquivo_destino}/{nome_arquivo}", imagem_tratada)

    arquivos = glob.glob(f'{arquivo_destino}/*')

    for arquivo in arquivos:
        imagem = Image.open(arquivo)
        imagem = prepare_image(imagem)
        imagem = remove_noise(imagem, noise)
        imagem2 = Image.new("P", imagem.size, 255)

        for x in range(imagem.size[1]):
            for y in range(imagem.size[0]):
                cor_pixel = imagem.getpixel((y,x))
                if cor_pixel < 127:
                    imagem2.putpixel((y,x), 0)

        for x in range(imagem2.size[1]):
            for y in range(imagem2.size[0]):
                cor_pixel = imagem2.getpixel((y,x))
                if cor_pixel == 255:
                    imagem2.putpixel((y,x), 0)
                else:
                    imagem2.putpixel((y,x), 255)

        if imagem2.mode != 'RGB':
            imagem2 = imagem2.convert('RGB')
        imagem2.save(arquivo)

if __name__ == "__main__":
    tratar_imagem(90, "/content/imagens", arquivo_destino="/content/imagens_tratadas")

In [ ]:
## SEPARAR AS LETRAS DE CADA CAPTCHA
def separar_letras(area_ret, tratado_file, origem, letter, calculo, metodo):
    arquivos = glob.glob(f"{tratado_file}/*")

    for arquivo in arquivos:
        imagem = cv2.imread(arquivo)
        imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2GRAY)
        _, imagem = cv2.threshold(imagem, 0, 255, cv2.THRESH_BINARY_INV)

        contornos, _ = cv2.findContours(imagem, calculo, metodo)

        #for i, contour in enumerate(contornos):
        #   x, y, w, h = cv2.boundingRect(contour)
        #  cv2.rectangle(imagem, (x, y), (x + w, y + h), (255, 0, 0), 1)
        regiao_letras = []

        for i,contorno in enumerate(contornos):

            (x, y, l, a) = cv2.boundingRect(contorno)
            area = cv2.contourArea(contorno)
            if area > area_ret and x > 0 and y > 0:
                regiao_letras.append((x,y,l,a))


        imagem_final = cv2.merge([imagem] * 3)
        i=0

        for retangulo in regiao_letras:
            x,y, largura, altura = retangulo
            imagem_letra = imagem[y-2:y+altura + 2, x-1:x+largura+1]
            i+=1
            nome_arquivo = os.path.basename(arquivo).replace(".png", f"letra{i}.png")
            cv2.imwrite(f"./{letter}/{nome_arquivo}", imagem_letra)
            cv2.rectangle(imagem_final, (x-2, y-2), (x + largura + 2, y + altura + 2), (255, 0, 0), 1)
        nome_arquivo = os.path.basename(arquivo)
        cv2.imwrite(f"./{origem}/{nome_arquivo}", imagem_final)

if __name__ == "__main__":

    separar_letras(90, "imagens_tratadas", "imagens_identificadas", "imagens_letras", cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [ ]:
##  - RECSIZE_TO_FIT

def resize_to_fit(image, width, height):

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image


In [ ]:
from keras.src.callbacks import History
##  - TREINAR

import matplotlib.pyplot as plt

def treinar(number_options, letter_file, nome_ia, nome_modelo):

    dados = []
    rotulos = []
    pasta_base_imagens = letter_file

    imagens = paths.list_images(pasta_base_imagens)

    for arquivo in imagens:
        rotulo = arquivo.split(os.path.sep)[-2]
        imagem = cv2.imread(arquivo)
        imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
        imagem = resize_to_fit(imagem, 20, 20)
        imagem = np.expand_dims(imagem, axis=2)

        dados.append(imagem)
        rotulos.append(rotulo)

    dados = np.array(dados, dtype="float") / 255
    rotulos = np.array(rotulos)

    # random_state(0) mesma divisão de dados se rodar novamente o mdeolo (garante consistencia de dados
    # em diferentes configurações)
    (X_train, X_test, Y_train, Y_test) = train_test_split(dados, rotulos, test_size=0.35, random_state=0)

    lb = LabelBinarizer().fit(Y_train)
    Y_train = lb.transform(Y_train)
    Y_test = lb.transform(Y_test)

    with open(nome_ia, 'wb') as arquivo_pickle:
        pickle.dump(lb, arquivo_pickle)

    #rede neural sequencial (fluxo em em pilha)
    model = Sequential()

    # (20,(5,5)) [= numero de filtros (20) ou mapas de caracteristicas diferentes
    # (5,5) é o tamanho dos filtros
    # padding = define o preenchimento com mesmas dimensoes (largura e altura) de entrada
    # (20,20,1) imagem com 20 pixels e 1 canal (escala de cinza)
    # introduzir não linearidade
    model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20,20,1), activation="relu"))

    # reduz dimensinalidade matendo caracteristicas mais importantes
    # (2,2) tamanho da janela
    # strides(movimento da janela horizontal e vertical) ajuda a não "overfitar"
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # (50, (5,5)) qtde de filtros (20) e tamanho do kernel (janelas da imagem)
    # padding tipo de preenchimento same =mesma dimensão da entrada
    model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))

    #
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # transforma saida em vetor unidimensional
    model.add(Flatten())

    # conecta todos os neuronios conectados a neuronios da camada anterior
    model.add(Dense(500, activation="relu"))

    # softmax (classificacao multiclasse)
    model.add(Dense(number_options, activation="softmax"))

    # categorical_crossentropy (calcula previsoes dos rotulos do modelo e reais)
    # otimizados ADAM (variante do gradiente descendente), atualiza pesos em funcao de perda
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    epochs = 50

    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=number_options, epochs=epochs, verbose=1)

    fig, ax = plt.subplots(1,2, figsize=(16,8))
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    legend = ax[1].legend(loc='best', shadow=True)

    model.save(nome_modelo)

if __name__ == "__main__":
    treinar(57, "letra", "rotulos_modelo_aula.dat", "modelo_treinado_aula.hdf5")

In [ ]:
##  - RESOLVER CAPTCHA

# noise=qtde ruidos (quanto maior mais dificil reconhecer)
def quebrar_captcha(noise, areas, ia_file, model_file, calculo, metodo):
    with open(ia_file, "rb") as arquivo_tradutor:
        lb = pickle.load(arquivo_tradutor)

    modelo = load_model(model_file)

    arquivos = glob.glob("resolver/*")

    tratar_captcha = tratar_imagem(noise, "resolver", arquivo_destino="resolver")
    arquivos = list(paths.list_images("resolver"))

    for arquivo in arquivos:
        imagem = cv2.imread(arquivo)
        imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2GRAY)
        _, imagem = cv2.threshold(imagem, 0, 255, cv2.THRESH_BINARY_INV)

        contornos, _ = cv2.findContours(imagem, calculo, metodo)

        regiao_letras = []
        previsao = []

        for i,contorno in enumerate(contornos):

            (x, y, l, a) = cv2.boundingRect(contorno)
            area = cv2.contourArea(contorno)

            if area > areas and x > 0 and y > 0:

                # posicao x,y, largura e altura
                regiao_letras.append((x,y,l,a))

        regiao_letras = sorted(regiao_letras, key=lambda x: x[0])
        imagem_final = cv2.merge([imagem] * 3)
        i=0

        for retangulo in regiao_letras:

            x,y, largura, altura = retangulo
            imagem_letra = imagem[y-2:y+altura + 2, x-1:x+largura+1]

            imagem_letra = resize_to_fit(imagem_letra, 20, 20)

            imagem_letra = np.expand_dims(imagem_letra, axis=2)
            imagem_letra = np.expand_dims(imagem_letra, axis=0)

            letra_prevista = modelo.predict(imagem_letra)
            letra_prevista = lb.inverse_transform(letra_prevista)[0]
            teste_letra = letra_prevista.split('_')
            if len(teste_letra) == 2:
                previsao.append(str(teste_letra[0]).upper())
            else:
                previsao.append(teste_letra[0])

        texto_previsao = "".join(previsao)
        print(texto_previsao)
        return texto_previsao

if __name__ == "__main__":
    ia_file = "rotulos_modelo_aula.dat"
    model_file = "modelo_treinado_aula.hdf5"
    quebrar_captcha(10, 20, ia_file, model_file, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)